In [ ]:
'''
!pip install mediapipe
!pip install seaborn
!pip install tensorflow
!pip install openpyxl
'''

'\n!pip install mediapipe\n!pip install seaborn\n!pip install tensorflow\n'

In [1]:
import mediapipe as mp
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import os
import shutil
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import glob
import cv2
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dense
from keras.callbacks import EarlyStopping
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
np.random.seed(42)
#RUN

In [ ]:
# Create a object from Holistic to detect (pose, face, and hands keypoints)
mp_holistic = mp.solutions.holistic
# Drawing utilities
mp_drawing = mp.solutions.drawing_utils

In [ ]:
'''
The input image is converted from BGR to RGB because the MediaPipe model expects RGB images.
It sets image.flags.writeable to False before passing it to the model to prevent any unwanted modifications during the inference process.
The images will converted back from RGB to BGR for any further OpenCV operations,
this ensures the image can be processed further by OpenCV without any issues related to color formats.
'''

def mediapipe_detection(image, model):

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
'''
This function is responsible for drawing the detected landmarks on the image, allowing you to imagen the pose and hand landmarks detected by MediaPipe.

The function draws the pose landmarks, left hand landmarks, and right hand landmarks with different colors and styles.

color: Defines the color of the landmarks.
thickness: Defines how thick the lines connecting the landmarks will be.
circle_radius: Controls the radius of the circles around each landmark.

This will makes it easier to detected landmarks on the image for debugging or interpretation purposes.
'''

def draw_styled_landmarks(image, results):

    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [ ]:
'''
This function adjusts the landmarks by centering them around a specific point LikeTHE ( the nose or wrist)
that's to normalize the positions and remove any translation shift in the data.

Reshape>> for thhe landmark array is reshaped to handle 3D points (x, y, z).

Centering>> Like (nose or wrist) is subtracted from each landmark to "center" the landmarks around the specific point.
It's look like Normalization :)
'''

def adjust_landmarks(arr,center):

    # Reshape the array to have shape (n, 3)
    arr_reshaped = arr.reshape(-1, 3)

    # Repeat the center array to have shape (n, 3)
    center_repeated = np.tile(center, (len(arr_reshaped), 1))

    # Subtract the center array from the arr array
    arr_adjusted = arr_reshaped - center_repeated

    # Reshape arr_adjusted back to shape (n*3,)
    arr_adjusted = arr_adjusted.reshape(-1)
    return(arr_adjusted)

In [ ]:
'''
This function extracts and adjusts keypoints:
Pose, Left Hand, Right Hand Keypoints: Each set of landmarks (pose, left hand, right hand) is flattened into a 1D array.
Also the landmarks are adjusted by centering around specific points (nose for pose, wrists for hands).
So. it's to extract and adjust the keypoints for each frame, making them ready for further analysis or machine learning models.
'''

def extract_keypoints(results):

    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    nose=pose[:3]
    lh_wrist=lh[:3]
    rh_wrist=rh[:3]
    pose_adjusted = adjust_landmarks(pose,nose)
    lh_adjusted = adjust_landmarks(lh,lh_wrist)
    rh_adjusted = adjust_landmarks(rh,rh_wrist)
    return pose_adjusted, lh_adjusted, rh_adjusted

In [8]:
'''
I have to adjust the range to fit the words and numbers :) #############
'''
# RUN
# Define the different ranges that needed
ranges = [(1, 32), (71, 503)]  # (1, 32) represents numbers, (71, 503) represents Words

# Initialize an empty list to store the results
selected_words = []

# Iterate over each range
for start, end in ranges:
    # Extend the list with zero-padded numbers in the current range
    selected_words.extend([str(num).zfill(4) for num in range(start, end)])

# Print the result
print(selected_words)

['0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009', '0010', '0011', '0012', '0013', '0014', '0015', '0016', '0017', '0018', '0019', '0020', '0021', '0022', '0023', '0024', '0025', '0026', '0027', '0028', '0029', '0030', '0031', '0071', '0072', '0073', '0074', '0075', '0076', '0077', '0078', '0079', '0080', '0081', '0082', '0083', '0084', '0085', '0086', '0087', '0088', '0089', '0090', '0091', '0092', '0093', '0094', '0095', '0096', '0097', '0098', '0099', '0100', '0101', '0102', '0103', '0104', '0105', '0106', '0107', '0108', '0109', '0110', '0111', '0112', '0113', '0114', '0115', '0116', '0117', '0118', '0119', '0120', '0121', '0122', '0123', '0124', '0125', '0126', '0127', '0128', '0129', '0130', '0131', '0132', '0133', '0134', '0135', '0136', '0137', '0138', '0139', '0140', '0141', '0142', '0143', '0144', '0145', '0146', '0147', '0148', '0149', '0150', '0151', '0152', '0153', '0154', '0155', '0156', '0157', '0158', '0159', '0160', '0161', '0162', '0163', '0164',

In [ ]:
'''
This function processes all the video frames of sign language sings for a particular signer and split (train, test, or val).
It does the following:

For each word, it processes video files.

For each frame in the video, the function:
>> Reads the frame.

>> Uses mediapipe_detection to detect landmarks.

>> Extracts the keypoints (pose, left hand, right hand).

>> Appends the keypoints to the respective lists.

>> After processing a video, the keypoints are saved as .npy files.


This function generates numpy arrays of keypoints for each video in the specified folder location.
Args:
    signer(int): the signer of interest. Could be 01 or 02 or 03
    split(str): can be 'train', 'test' or 'val'
'''
def make_keypoint_arrays(path,signer,split):

    os.makedirs('karsl-502',exist_ok = True)
    os.makedirs(f'karsl-502/{signer}',exist_ok = True)
    os.makedirs(f'karsl-502/{signer}/{split}',exist_ok = True)
    working_path = f'karsl-502/{signer}/{split}'
    words_folder = os.path.join(path,str(signer),str(signer), split)

    # Loop through all the subfolders in the folder
    for word in tqdm(selected_words):

        video_files = os.listdir(os.path.join(words_folder, word))
          # Loop through the video files
        for video_file in video_files:
                # Open the video file
            video = sorted(os.listdir(os.path.join(words_folder, word, video_file)))

            # Initialize the list of keypoints for this video
            pose_keypoints, lh_keypoints, rh_keypoints = [], [], []
            with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
              # Loop through the video frames
              for frame in video:
                  # Perform any necessary preprocessing on the frame (e.g., resizing, normalization)
                frame = os.path.join(words_folder, word, video_file,frame)
                frame = cv2.imread(frame)
#                 frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                  # Normalize pixel values to the range [0, 1]
                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Extract keypoints
                pose, lh, rh = extract_keypoints(results)
                # Add the keypoints to the list for this video
                pose_keypoints.append(pose)
                lh_keypoints.append(lh)
                rh_keypoints.append(rh)
                # Save the keypoints for this video to a numpy array
                pose_directory = os.path.join(working_path, word,'pose_keypoints')
                lh_directory = os.path.join(working_path, word,'lh_keypoints')
                rh_directory = os.path.join(working_path, word,'rh_keypoints')

                if not os.path.exists(pose_directory):
                    os.makedirs(pose_directory)

                if not os.path.exists(lh_directory):
                    os.makedirs(lh_directory)

                if not os.path.exists(rh_directory):
                    os.makedirs(rh_directory)

                pose_path = os.path.join(pose_directory, video_file)
                np.save(pose_path, pose_keypoints)

                lh_path = os.path.join(lh_directory, video_file)
                np.save(lh_path, lh_keypoints)

                rh_path = os.path.join(rh_directory, video_file)
                np.save(rh_path, rh_keypoints)

In [ ]:
#make_keypoint_arrays('G:\Capstone data\karsl-502','01','train')
#make_keypoint_arrays('G:\Capstone data\karsl-502','01','test')
#make_keypoint_arrays('G:\Capstone data\karsl-502','02','train')
#make_keypoint_arrays('G:\Capstone data\karsl-502','02','test')
#make_keypoint_arrays('G:\Capstone data\karsl-502','03','train')
#make_keypoint_arrays('G:\Capstone data\karsl-502','03','test')

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 463/463 [4:05:33<00:00, 31.82s/it]
